In [ ]:
import warnings
from utils import _parse_template
plt.close('all')

figures = []
figures_no_labels = []
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for spec in figure_specs:
        if spec['show_voltage']:
            fig, axs = plt.subplots(len(height_ratios),1, figsize=figsize, sharex=True, gridspec_kw={'height_ratios': height_ratios})
            fig_no_labels, axs_no_labels = plt.subplots(len(height_ratios),1, figsize=figsize, sharex=True, gridspec_kw={'height_ratios': height_ratios})
        else:
            fig, axs = plt.subplots(1,1, figsize=figsize)
            fig_no_labels, axs_no_labels = plt.subplots(1,1, figsize=figsize)
            axs = [None, axs]
            axs_no_labels = [None, axs_no_labels]
        if spec['sub_selection'] is None:
            spec['sub_selection'] = list(range(len(selection_means)))
        sel_means = [selection_means[i] for i in spec['sub_selection']]
        sel_stds = [selection_stds[i] for i in spec['sub_selection']]
        sel_raw = [selection_raw[i] for i in spec['sub_selection']]
        for i, (id_mean, id_std, id_raw) in enumerate(zip(sel_means, sel_stds, sel_raw)):
            data_mean = an.measurements[id_mean]['data']
            data_std = an.measurements[id_std]['data']  
            data_raw = an.measurements[id_raw]['data'] 
            x = data_mean[spec['x_axis']]
            y = data_mean[spec['y_axis']]
            y_err = data_std[spec['y_axis']]
            x_raw = data_raw[spec['x_axis']]
            y_raw = data_raw['Voltage']
            if 'abs' in spec and spec['abs']:
                y = y.abs()
            if spec['xlim'].size:
                mask = (x >= spec['xlim'][0]) & (x <= spec['xlim'][1])
                mask_raw = (x_raw >= spec['xlim'][0]) & (x_raw <= spec['xlim'][1])
                x = x[mask]
                y = y[mask]
                y_err = y_err[mask]
                x_raw = x_raw[mask_raw]
                y_raw = y_raw[mask_raw]
            #axs.errorbar(x, y, y_err, linestyle='None', marker='o', markersize=2, capsize=2)
            idx = spec['sub_selection'][i]
            color = color_map(cmap, idx)
            if spec['errorbar']:
                axs[1].errorbar(x, y*y_factor, yerr=y_err*y_factor, color=color)
                axs_no_labels[1].errorbar(x, y*y_factor, yerr=y_err*y_factor, color=color)
            else:
                axs[1].plot(x, y*y_factor, color=color, linestyle=linestyle_map[i])
                axs[1].fill_between(x, (y-y_err)*y_factor, (y+y_err)*y_factor, color=color, alpha=0.5)
                axs_no_labels[1].plot(x, y*y_factor, color=color, linestyle=linestyle_map[i]);
                axs_no_labels[1].fill_between(x, (y-y_err)*y_factor, (y+y_err)*y_factor, color=color, alpha=0.5);
            if spec['show_voltage']:
                axs[0].plot(x_raw, y_raw, color=color, linestyle=linestyle_map[i])
                axs_no_labels[0].plot(x_raw, y_raw, color=color, linestyle=linestyle_map[i])
        
        if legend_labels:
            if spec['errorbar']:
                labels = legend_labels
            else:
                labels = sum([[legend_labels[i], '_no_label_'] for i in spec['sub_selection']], [])
            axs[1].legend(labels)
        axs[1].set_xlabel(spec['x_axis'] + ' ' + units[spec['x_axis']])
        axs[1].set_ylabel(spec['y_axis'] + ' ' + units[spec['y_axis']])
        
        name = _parse_template(an.measurements[sel_means[0]], spec['title'] + title_appendix + ' ({cluster} {device})')
        if spec['show_voltage']:
            axs[0].set_title(name)
            axs[0].grid()
            axs[0].set_xscale(spec['xscale'])
            axs[0].set_yscale(spec['yscale'])
            axs_no_labels[0].set_xscale(spec['xscale'])
            axs_no_labels[0].set_yscale(spec['yscale'])
            if spec['xlim'].size:
                axs[0].set_xlim(tuple(spec['xlim']))
                axs_no_labels[0].set_xlim(tuple(spec['xlim']))
            if spec['ylim_raw'].size:
                axs[0].set_ylim(tuple(spec['ylim_raw']))
                axs_no_labels[0].set_ylim(tuple(spec['ylim_raw']))
        else:
            axs[1].set_title(name)
        
        
        axs[1].grid()

        axs[1].set_xscale(spec['xscale'])
        axs[1].set_yscale(spec['yscale'])
        axs_no_labels[1].set_xscale(spec['xscale'])
        axs_no_labels[1].set_yscale(spec['yscale'])
        if spec['xlim'].size:
            axs[1].set_xlim(tuple(spec['xlim']))
            axs_no_labels[1].set_xlim(tuple(spec['xlim']))
        if spec['ylim'].size:
            axs[1].set_ylim(tuple(spec['ylim'] * y_factor))
            axs_no_labels[1].set_ylim(tuple(spec['ylim'] * y_factor))
        fig.tight_layout()
        fig_no_labels.tight_layout()
        if spec['show_voltage']:
            fig.subplots_adjust(hspace=h_space)
            fig_no_labels.subplots_adjust(hspace=h_space)
        figures.append(fig)
        figures_no_labels.append(fig_no_labels)